# Utilisation de ffmpeg dans le projet Meteorix

### Objectif

Notre objectif est d'adresser une puce matérielle pour encoder une vidéo à partir d'images brutes. En effet, sur le nano-satellite nous aurons uniquement une puce pour faire l'encodage de la vidéo. Pour tester cela nous allons utiliser FFMPEG avec différentes options pour voir comment sont encodés les images.

### 1.Transformer une vidéo compressé au format ppm

Pour le moment nous avons des vidéos dans des formats compressés, or la caméra du satellite récupère des images au format ppm. Nous voulons donc transformer nos vidéos compressés en image ppm pour ensuite donner ces image à la puce pour qu'elle les compresse.

La commande suivante permet de le faire : mkdir -p output_images && ffmpeg -i video.mpg output_images/image%d.ppm

Elle stocke aussi toutes les vidéos dans un dossier output

### 2. Copier les images sur la machine EM780

Nous allons copier les vidéos sur une le disque dur d'une machine qui à les encodeurs similaires à celle que l'on a sur le satellite.

Pour se connecter à la machine : ssh em780.mono.lip6.ext

Il faut faire les commandes suivantes :
- (en local) scp le-fichier-sur-le-mac front.mono.lip6.ext:~/
- (sur em780) mv v03.mp4 /scratch/renaudc-nfs
Puis on va dans notre dossier cd /scratch/renaudc-nfs pour travailler sur nos dossiers

De plus, FFMPEG est déjà installé sur la machine, on pourra donc transférer la vidéo et faire les commandes FFMPEG depuis la machine.

### 3.Encoder les images ppm
- adresser la puce sur le GPU embarqué AMD Radeon 780M
- adresser la puce sur le GPU externe NVidia GeForce RTX 3090 (dans ce cas il faut utiliser NVENC)

L'option 1 est la meilleure car la puce est sur le SoC, ce qui sera probablement plus proche de la réalité dans un nano-satellite.
L'option 2 serai certainement plus facile car NVENC est bien supporté et documenté en général

On peut encoder avec la commande : ffmpeg -hwaccel cuda -framerate 30 -i output_images/image%d.ppm -c:v h264_nvenc -preset fast -b:v 5M compressed_video.mp4 -loglevel verbose

-hwaccel cuda : Active l'accélération matérielle CUDA pour optimiser le traitement.
-c:v h264_nvenc : Utilise NVENC pour encoder la vidéo au format H.264.

On peut stocker l'utilisation du GPU avec la première commande
nohup watch -n 0.1 nvidia-smi > gpu_usage.log & 
ffmpeg -hwaccel cuda -i output_images/image%d.ppm -c:v h264_nvenc -preset fast compressed_video.mp4

